# 네이버 영화 리뷰 감성 분류

IMDb 영화 리뷰 데이터셋과 비슷한 네이버 영화 리뷰 데이터셋(https://github.com/e9t/nsmc)을 사용해 한글 문장의 감성 분류 예제를 다루어 보겠습니다. 이 데이터는 네이버 영화 사이트에 있는 리뷰 20만 개를 모은 것입니다. 네이버 영화 리뷰 데이터셋 깃허브에서 직접 데이터를 다운로드 받아도 되지만 편의를 위해 이 책의 깃허브의 `ch09` 폴더에 데이터셋을 넣어 놓았습니다.

20만개 데이터 중 15만개는 훈련 데이터셋으로 `ratings_train.txt` 파일에 저장되어 있고 5만개는 테스트 데이터셋으로 `ratings_test.txt` 파일에 저장되어 있습니다. 리뷰의 길이는 140을 넘지 않습니다. 부정 리뷰는 1\~4까지 점수를 매긴 리뷰이고 긍정 리뷰는 6\~10까지 점수를 매긴 리뷰입니다. 훈련 데이터셋과 테스트 데이터셋의 부정과 긍정 리뷰는 약 50%씩 구성되어 있습니다.

한글은 영어와 달리 조사와 어미가 발달해 있기 때문에 BoW나 어간 추출보다 표제어 추출 방식이 적합합니다. 이런 작업을 형태소 분석이라 부릅니다. 파이썬에서 한글 형태소 분석을 하기 위한 대표적인 패키지는 `konlpy`와 `soynlp`입니다. 두 패키지를 모두 사용해 네이버 영화 리뷰를 긍정과 부정으로 분류해 보겠습니다.

먼저 이 예제를 실행하려면 `konlpy`와 `soynlp`가 필요합니다. 다음 명령을 실행해 두 패키지를 설치해 주세요.

In [ ]:
!pip install konlpy soynlp

그다음 `konlpy`, `pandas`, `numpy`를 임포트합니다.

In [1]:
import konlpy
import pandas as pd
import numpy as np

코랩을 사용하는 경우 다음 코드 셀의 주석을 제거하고 실행하세요.

In [ ]:
#!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_train.txt -O ratings_train.txt

감성 분류를 시작하기 전에 훈련 데이터셋과 테스트 데이터셋을 각각 판다스 데이터프레임으로 읽은 후 넘파이 배열로 준비하겠습니다. 먼저 훈련 데이터셋부터 읽어 보죠. `ratings_train.txt` 파일은 하나의 리뷰가 한 행을 구성하며 각 필드는 탭으로 구분되어 있기 때문에 판다스의 `read_csv()` 함수로 간편하게 읽어 들일 수 있습니다. `read_csv()`는 기본적으로 콤마를 기준으로 필드를 구분하므로 `delimiter='\t'`으로 지정하여 탭으로 변경합니다. 기본적으로 판다스는 빈 문자열을 NaN으로 인식합니다. 빈 문자열을 그대로 유지하기 위해 `keep_default_na` 매개변수를 `False`로 지정합니다.

In [2]:
df_train = pd.read_csv('ratings_train.txt', 
                       delimiter='\t', keep_default_na=False)

데이터프레임의 `head()` 메서드를 호출하면 처음 5개의 행을 출력해 줍니다. 이 예제에서 사용할 데이터는 document 열과 label 열입니다. label은 리뷰가 긍정(1)인지 부정(0)인지를 나타내는 값입니다.

In [3]:
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


데이터프레임의 열을 선택하여 `Series` 객체의 `values` 속성을 사용하면 document 열과 label 열을 넘파이 배열로 저장할 수 있습니다. 각각 훈련 데이터셋의 특성과 타깃 값으로 저장합니다.

In [4]:
X_train = df_train['document'].values
y_train = df_train['label'].values

코랩을 사용하는 경우 다음 코드 셀의 주석을 제거하고 실행하세요.

In [ ]:
#!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_test.txt -O ratings_test.txt

`ratings_test.txt` 파일에 대해서도 동일한 작업을 수행합니다.

In [5]:
df_test = pd.read_csv('ratings_test.txt', 
                      delimiter='\t', keep_default_na=False)

X_test = df_test['document'].values
y_test = df_test['label'].values

훈련 데이터셋과 테스트 데이터셋의 크기를 확인해 보죠. 각각 150,000개와 50,000개의 샘플을 가지고 있고 양성 클래스와 음성 클래스의 비율은 거의 50%에 가깝습니다.

In [6]:
print(len(X_train), np.bincount(y_train))

150000 [75173 74827]


In [7]:
print(len(X_test), np.bincount(y_test))

50000 [24827 25173]


훈련 데이터셋과 테스트 데이터셋을 준비했으므로 형태소 분석기를 사용해 본격적인 감성 분류 작업을 시작해 보겠습니다.

`konlpy`는 5개의 한국어 형태소 분석기를 파이썬 클래스로 감싸서 제공하는 래퍼 패키지입니다. `konlpy`가 제공하는 형태소 분석기에 대한 자세한 내용은 온라인 문서(https://konlpy.org/ko/latest/)를 참고하세요. 이 예에서는 스칼라로 개발된 open-korean-text 한국어 처리기(https://github.com/open-korean-text/open-korean-text)를 제공하는 `Okt` 클래스를 사용해 보겠습니다. open-korean-text는 비교적 성능이 높고 별다른 설치 없이 구글 코랩에서도 바로 사용할 수 있습니다.

`konlpy.tag` 패키지에서 `Okt` 클래스를 임포트하고 객체를 만든 다음 훈련 데이터셋에 있는 문장 하나를 `morphs()` 메서드로 형태소로 나누어 보겠습니다.

In [8]:
from konlpy.tag import Okt

okt = Okt()
print(X_train[4])
print(okt.morphs(X_train[4]))

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']


한글 문장에서 조사와 어미가 잘 구분되어 출력된 것을 볼 수 있습니다. '사이몬페그'와 같은 고유 명사는 처리하는데 어려움을 겪고 있네요. 완벽하지는 않지만 이 클래스를 사용해 분류 문제를 풀어 보겠습니다.

`TfidfVectorzier`는 기본적으로 공백을 기준으로 토큰을 구분하지만 `tokenizer` 매개변수에 토큰화를 위한 사용자 정의 함수를 전달할 수 있습니다. 따라서 앞서 테스트했던 `okt.morphs` 메서드를 전달하면 형태소 분석을 통해 토큰화를 수행할 수 있습니다. `tokenizer` 매개변수를 사용할 때 패턴`token_pattern=None`으로 지정하여 `token_pattern` 매개변수가 사용되지 않는다는 경고 메시지가 나오지 않게 합니다.

`TfidfVectorzier`을 `ngram_range=(1, 2)`로 설정하여 유니그램과 바이그램을 사용하고 `min_df=3`으로 지정하여 3회 미만으로 등장하는 토큰은 무시합니다. 또한 `max_df=0.9`로 두어 가장 많이 등장하는 상위 10%의 토큰도 무시하겠습니다. 이런 작업이 불용어로 생각할 수 있는 토큰을 제거할 것입니다.

컴퓨팅 파워가 충분하다면 하이퍼파라미터 탐색 단계에서 `TfidfVectorzier`의 매개변수도 탐색해 보는 것이 좋습니다. 여기에서는 임의의 매개변수 값을 지정하여 데이터를 미리 변환하고 하이퍼파라미터 탐색에서는 분류기의 매개변수만 탐색하겠습니다.

노트: 토큰 데이터를 생성하는 시간이 많이 걸리므로 주피터 노트북에서는 다음 번 실행 때 이 과정을 건너 뛸 수 있도록 이 데이터를 한 번 생성하여 npz 파일로 저장합니다.

In [10]:
import os
from scipy.sparse import save_npz, load_npz
from sklearn.feature_extraction.text import TfidfVectorizer

if not os.path.isfile('okt_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2), 
                            min_df=3,
                            max_df=0.9,
                            tokenizer=okt.morphs, 
                            token_pattern=None)
    tfidf.fit(X_train)
    X_train_okt = tfidf.transform(X_train)
    X_test_okt = tfidf.transform(X_test)
    save_npz('okt_train.npz', X_train_okt)
    save_npz('okt_test.npz', X_test_okt)
else:
    X_train_okt = load_npz('okt_train.npz')
    X_test_okt = load_npz('okt_test.npz')

`X_train_okt`와 `X_test_okt`가 준비되었으므로 `SGDClassifier` 클래스를 사용해 감성 분류 문제를 풀어 보겠습니다. 탐색할 `SGDClassifier`의 매개변수는 규제를 위한 `alpha` 매개변수입니다. `RandomizedSearchCV` 클래스를 사용하기 위해 `loguniform` 함수로 탐색 범위를 지정하겠습니다. 여기에서는 `SGDClassifier`의 손실 함수로 로지스틱 손실(`'log'`)을 사용하지만 다른 손실 함수를 매개변수 탐색에 포함할 수 있습니다. 총 반복 회수(`n_iter`)는 50회로 지정합니다. 만약 CPU 코어가 여러개라면 `n_jobs` 매개변수를 1 이상으로 설정하여 수행 속도를 높일 수 있습니다.

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform

sgd = SGDClassifier(loss='log', random_state=1)
param_dist = {'alpha': loguniform(0.0001, 100.0)}

rsv_okt = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50,
                             random_state=1,
                             verbose=1)
rsv_okt.fit(X_train_okt, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:  1.1min finished


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f313ea48908>},
                   random_state=1, verbose=1)

하이퍼파라미터 탐색으로 찾은 최상의 점수와 매개변수 값을 확인해 보죠.

In [12]:
print(rsv_okt.best_score_)
print(rsv_okt.best_params_)

0.8251533333333334
{'alpha': 0.0001001581395585897}


테스트 데이터셋 `X_test_okt`에서 점수도 확인해 보겠습니다.

In [13]:
rsv_okt.score(X_test_okt, y_test)

0.8189

약 82%의 정확도를 냈습니다. 간단한 작업으로 꽤 좋은 성능을 냈습니다. `konlpy`의 다른 형태소 분석 클래스를 사용하거나 `SGDClassifier` 외에 다른 분류기를 시도하지 않을 이유는 없습니다. 충분한 컴퓨팅 파워가 없다면 사이킷런 0.24버전에서 추가되는 `HalvingRandomSearchCV` 클래스를 사용해 볼 수도 있습니다.

이번에는 또 다른 파이썬 형태소 분석기인 `soynlp`를 사용해 보겠습니다. `soynlp`는 순수하게 파이썬으로 구현된 형태소 분석 패키지입니다. 깃허브(https://github.com/lovit/soynlp)에는 소스 코드 뿐만 아니라 다양한 튜토리얼도 함께 제공합니다. `soynlp`는 3개의 토큰화 클래스를 제공합니다. 기본적으로 띄어쓰기가 잘 되어 있다면 `LTokenizer`가 잘 맞습니다. 이외에는 `MaxScoreTokenizer`와 `RegexTokenizer`가 있습니다. 이 예에서는 `LTokenizer`를 사용해 보겠습니다. 먼저 `soynlp.tokenizer`에서 `LTokenizer`를 임포트합니다.

In [14]:
from soynlp.tokenizer import LTokenizer

`LTokenizer` 클래스의 객체를 만든다음 앞에서와 같이 훈련 데이터셋에 있는 샘플(`X_train[4]`) 하나의 형태소를 분석해 보겠습니다.

In [15]:
lto = LTokenizer()

print(lto.tokenize(X_train[4]))

['사이몬페그의', '익살스런', '연기가', '돋보였던', '영화!스파이더맨에서', '늙어보이기만', '했던', '커스틴', '던스트가', '너무나도', '이뻐보였다']


`soynlp`는 말뭉치의 통계 데이터를 기반으로 동작하기 때문에 기본 `LTokenizer` 객체로는 공백으로만 토큰화를 수행합니다. `LTokenizer`에 필요한 통계 데이터를 생성하기 위해서 `WordExtractor`를 사용해 보겠습니다.

In [16]:
from soynlp.word import WordExtractor

`WordExtractor` 객체를 만든 후 `train()` 메서드에 `X_train`을 전달하여 훈련합니다. 훈련이 끝나면 `word_scores()` 메서드에서 단어의 점수를 얻을 수 있습니다. 반환된 `scores` 객체는 단어마다 결합 점수(cohesion score)와 브랜칭 엔트로피(branching entropy)를 가진 딕셔너리입니다.

In [17]:
word_ext = WordExtractor()
word_ext.train(X_train)
scores = word_ext.word_scores()

training was done. used memory 1.737 Gb
all cohesion probabilities was computed. # words = 85683
all branching entropies was computed # words = 101540
all accessor variety was computed # words = 101540


`soynlp` 깃허브의 튜토리얼(https://github.com/lovit/soynlp/blob/master/tutorials/wordextractor_lecture.ipynb)을 따라 결합 점수(`cohesion_forward`)와 브랜칭 엔트로피(`right_branching_entropy`)에 지수를 취한 값에 곱하여 최종 점수를 만들겠습니다.

In [18]:
import math

score_dict = {key: scores[key].cohesion_forward *
              math.exp(scores[key].right_branching_entropy) 
              for key in scores}

이제 이 점수를 `LTokenizer`의 `scores` 매개변수로 전달하여 객체를 만들고 앞에서 테스트한 샘플에 다시 적용해 보겠습니다.

In [19]:
lto = LTokenizer(scores=score_dict)

In [20]:
print(lto.tokenize(X_train[4]))

['사이', '몬페그의', '익살스', '런', '연기', '가', '돋보', '였던', '영화', '!스파이더맨에서', '늙어', '보이기만', '했던', '커스틴', '던스트가', '너무', '나도', '이뻐', '보였다']


단어 점수를 활용했기 때문에 토큰 추출이 훨씬 잘 된 것을 볼 수 있습니다. `lto.tokenizer` 메서드를 `TfidVectorizer` 클래스에 전달하여 `konlpy`를 사용했을 때와 같은 조건으로 훈련 데이터셋과 테스트 데이터셋을 변환해 보겠습니다.

In [22]:
if not os.path.isfile('soy_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2),
                            min_df=3,
                            max_df=0.9,
                            tokenizer=lto.tokenize, 
                            token_pattern=None)
    tfidf.fit(X_train)
    X_train_soy = tfidf.transform(X_train)
    X_test_soy = tfidf.transform(X_test)
    save_npz('soy_train.npz', X_train_soy)
    save_npz('soy_test.npz', X_test_soy)
else:
    X_train_soy = load_npz('soy_train.npz')
    X_test_soy = load_npz('soy_test.npz')

동일한 `SGDClassifier` 객체와 매개변수 분포를 지정하고 하이퍼파라미터 탐색을 수행해 보겠습니다.

In [23]:
rsv_soy = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50,
                             random_state=1,
                             verbose=1)
rsv_soy.fit(X_train_soy, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:   54.6s finished


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f313ea48908>},
                   random_state=1, verbose=1)

`soynlp`를 사용했을 때 최상의 점수와 매개변수는 다음과 같습니다.

In [24]:
print(rsv_soy.best_score_)
print(rsv_soy.best_params_)

0.8141066666666665
{'alpha': 0.0001001581395585897}


마지막으로 테스트 데이터셋에 대한 점수를 확인해 보겠습니다.

In [25]:
rsv_soy.score(X_test_soy, y_test)

0.8085

`Okt`를 사용했을 때 보다는 조금 더 낮지만 약 81% 이상의 정확도를 얻었습니다. 